## Inspecting Data

In [1]:
import gen_input

import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

In [2]:
train_test_valid_split = [1., 0., 0.]
# From http://ufldl.stanford.edu/housenumbers/
svhn_train = gen_input.read_data_sets("data/train_32x32.mat", train_test_valid_split).train
svhn_test = gen_input.read_data_sets("data/test_32x32.mat", train_test_valid_split).train

print svhn_train.images[0].shape
print svhn_train.images.shape
print svhn_test.images[0].shape
print svhn_test.images.shape

(1024, 3)
(73257, 1024, 3)
(1024, 3)
(26032, 1024, 3)


In [3]:
# per_img_means = svhn_train.images.mean(axis=1)
# # per_img_norm = svhn_train.images - per_img_means[:, np.newaxis, :]

In [4]:
# for i in range(4010, 4025):
#     fig = plt.figure()
#     ax = fig.add_subplot(1,1,1)
    
#     img = svhn_train.images[i]
#     lbl = np.argmax(svhn_train.labels[i])
#     ax.set_title(lbl)
#     img = img.reshape([32,32,3])
#     imgplot = plt.imshow(img)
#     plt.show()
    
# #     img_s = per_img_norm[i]
# #     img_s = img_s.reshape([32,32,3])
# #     imgplot = plt.imshow(img_s)
# #     plt.show()

## Model

In [5]:
# Data information
input_channels = 3
image_size = 32
n_classes = 10 # total classes (0-9 digits)

In [6]:
##########################################
##                                      ##
##              Parameters              ##
##                                      ##
##########################################
# Training Parameters
learning_rate = 1e-3
training_epochs = 2 # <-- 2 for now
batch_size = 100 #
total_batches = int(0.025 * svhn_train.num_examples / batch_size) # <-- only 2.5% of data for now

test_every = 5 # record test accuracy every 5 batches
test_batch_size = int(0.025*svhn_test.num_examples) # <--- only 2.5% of the test batch for now
final_test_size = test_batch_size*5

# 1 - drop out 
train_keep_prob = 0.9

In [7]:
ts = datetime.now().strftime('%Y%m%d_%H%M')
logs_path = "logs/{}/".format(ts)

# RESET TF GRAPH, just in case
tf.reset_default_graph()

In [8]:
##########################################
##                                      ##
##            Helper Wrappers           ##
##                                      ##
##########################################

# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape, stddev=0.1):
  """Create a weight variable with appropriate initialization."""
  initial = tf.truncated_normal(shape, stddev)
  return tf.Variable(initial)

def bias_variable(shape, default_bias=0.1):
  """Create a bias variable with appropriate initialization."""
  initial = tf.constant(default_bias, shape=shape)
  return tf.Variable(initial)

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

def convlayer(layer_name, input_tensor, receptive_field, channels_in, channels_out,
              padding='SAME', stride=1, act=tf.nn.relu, 
              pool=True, pooler=tf.nn.max_pool, pool_size=2, pool_stride=2, pool_padding='SAME'):
  """General purpose convolutional layer, followed by pooling

  It does a matrix convolution, bias add, and then uses relu by default to nonlinearize.
  Then it pools using max pooling by default.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops for TensorBoard.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([receptive_field, receptive_field, channels_in, channels_out])
      variable_summaries(weights)
    with tf.name_scope('biases'):
      biases = bias_variable([channels_out])
      variable_summaries(biases)
    with tf.name_scope('W_conv_x_plus_b'):
      preactivate = tf.nn.conv2d(input_tensor, weights, 
                                 strides=[1, stride, stride, 1], 
                                 padding=padding) + biases
      tf.summary.histogram('pre_activations', preactivate)
      
    activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)
    
    if pool:
      max_pool = pooler(activations, ksize=[1, pool_size, pool_size, 1], 
                      strides=[1, pool_stride, pool_stride, 1],
                      padding=pool_padding)

      tf.summary.histogram('pools', max_pool)
      return max_pool
    else: 
      return activations
    
def nn_layer(layer_name, input_tensor, input_dim, output_dim, act=tf.nn.relu):
  """Reusable code for making a normal neural net layer.
  It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      variable_summaries(weights)
    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
      variable_summaries(biases)
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
      tf.summary.histogram('pre_activations', preactivate)
    activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)
    return activations

In [9]:
# Input placeholders
with tf.name_scope('input'):
  x = tf.placeholder(tf.float32, shape=[None, image_size*image_size, input_channels], name="x-input") 
  y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

with tf.name_scope('input_reshape'):
  image_shaped_input = tf.reshape(x, [-1, image_size, image_size, input_channels])
  tf.summary.image('input', image_shaped_input, 10)

In [10]:
# Architecture

conv1 = convlayer(layer_name='conv1', input_tensor=image_shaped_input, receptive_field=5, 
                    channels_in=input_channels, channels_out=16)

conv2 = convlayer(layer_name='conv2', input_tensor=conv1, receptive_field=5, 
                  channels_in=16, channels_out=32)

dim = 1 # Compute how many numbers we have, ignoring the batch size
for d in conv2.get_shape()[1:].as_list():
  dim *= d

with tf.name_scope('conv2_reshape'):
  conv_shaped = tf.reshape(conv2, [-1, dim])

fc1 = nn_layer(layer_name='fc1', input_tensor=conv_shaped, input_dim=dim, output_dim=64)

with tf.name_scope('dropout'):
  keep_prob = tf.placeholder(tf.float32)
  tf.summary.scalar('dropout_keep_probability', keep_prob)
  dropped = tf.nn.dropout(fc1, keep_prob)
  
# Do not apply softmax activation yet! use the identity
y = nn_layer(layer_name='output', input_tensor=dropped, input_dim=64, output_dim=10, act=tf.identity) 

In [11]:
with tf.name_scope('cross_entropy'):
  # The raw formulation of cross-entropy,
  #
  # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
  #                               reduction_indices=[1]))
  #
  # can be numerically unstable.
  #
  # So here we use tf.nn.softmax_cross_entropy_with_logits on the
  # raw outputs of the nn_layer above, and then average across
  # the batch.
  diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
  with tf.name_scope('total'):
    cross_entropy = tf.reduce_mean(diff)

tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
  train_step = tf.train.AdamOptimizer(learning_rate).minimize(
      cross_entropy)

with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

sess = tf.InteractiveSession()

# Merge all the summaries and write them out
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(logs_path + '/train', sess.graph)
test_writer = tf.summary.FileWriter(logs_path + '/test')

tf.global_variables_initializer().run()


# Normalize by subtracting per image, per channel means
def normalize_batch(batch):
  per_img_ch_means = batch.mean(axis=1)
  return batch - per_img_ch_means[:, np.newaxis, :]

# Train the model, and also write summaries.
# Every 10th step, measure test-set accuracy, and write test summaries
# All other steps, run train_step on training data, & add training summaries

def feed_dict(train):
  """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
  if train:
    batch_x, batch_y = svhn_train.next_batch(batch_size)
    batch_x = normalize_batch(batch_x)
    keep_proba = train_keep_prob
  else:
    batch_x, batch_y = svhn_test.next_batch(test_batch_size)
    keep_proba = 1.0
  return {x: batch_x, y_: batch_y, keep_prob: keep_proba}

In [12]:
for epoch in xrange(training_epochs):
  for batch_num in xrange(total_batches):
    if batch_num % test_every == 0 and batch_num > 0:  
      # Record summaries and accuracy on the *test* set
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(train=False))
      test_writer.add_summary(summary, epoch * total_batches + batch_num)
      print('Accuracy at epoch %s - step %s: %s' % (epoch, batch_num, acc))
    else:  # Record train set summaries, and train
      if batch_num% 10 == 9:  
        # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step{}'.format(epoch * total_batches + batch_num))
        train_writer.add_summary(summary, epoch * total_batches + batch_num)
        print('Adding run metadata for', epoch * total_batches + batch_num)
      else:  
        # Record a normal summary on training data
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, epoch * total_batches + batch_num)
  
print "\nOptimization Finished!\n"
print "-"* 70
pwd = os.getcwd()+"/"
print("Run the following to start tensorboard server:\n" \
      "tensorboard --logdir=/{}{}".format(pwd, logs_path))

train_writer.close()
test_writer.close()

Accuracy at epoch 0 - step 5: 0.184615
('Adding run metadata for', 9)
Accuracy at epoch 0 - step 10: 0.149231
Accuracy at epoch 0 - step 15: 0.173846
Accuracy at epoch 1 - step 5: 0.161538
('Adding run metadata for', 27)
Accuracy at epoch 1 - step 10: 0.127692
Accuracy at epoch 1 - step 15: 0.158462

Optimization Finished!

----------------------------------------------------------------------
Run the following to start tensorboard server:
tensorboard --logdir=//home/andy/Desktop/cnn_svhn/logs/20170421_2034/


In [13]:
# with tf.name_scope('Optimizer'):
#     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
#     # Op to calculate every variable gradient
#     grads = tf.gradients(loss, tf.trainable_variables())
#     grads = list(zip(grads, tf.trainable_variables()))
    
#     # Op to update all variables according to their gradient
#     apply_grads = optimizer.apply_gradients(grads_and_vars=grads)

# # Summaries to visualize gradients
# for grad, var in grads:
#     var_name = var.name.replace(":", "_") # to suppress the pesky warning
#     tf.summary.histogram(var_name + '_gradient', grad)